In [1]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Análisis de personas distintas atendidas por diagnóstico

En este cuadernillo se quieren responder las siguiente preguntas:

1. ¿Cuántas personas fueron atendidas para cada uno de los diagnósticos del INT?

In [13]:
# Lee solamente la base del INT
ANIOS_A_ANALIZAR = [2015, 2016, 2017, 2018, 2019]

df_torax = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(
        (pl.col("ESTABLECIMIENTO_SALUD") == 112103) & (pl.col("ANO_EGRESO").is_in(ANIOS_A_ANALIZAR))
    )
    .collect(streaming=True)
)

Los problemas al momento de calcular la cantidad de pacientes distintos para un problema de salud son los siguientes:

1. Un paciente puede tener egresos hospitalarios con distintos diagnosticos que apuntan a un mismo problema de salud (Ej: Diagnostico E840, E841, E849 apuntar al problema de salud "Fibrosis Quistica"). Esto es un problema, ya que el paciente sera considerado como un paciente distinto dependiendo del diagnostico asignado. A modo de ilustracion, se tiene el siguiente ejemplo:

    - Paciente X diagnosticado con E840 y E841 en un mismo año. Esto significaría que hay 1 paciente distinto para E840 y E841. Sin embargo, si analizamos el problema de salud "Fibrosis Quística", solamente existiría 1 paciente distinto (y no 2).

Debido a lo anterior, es necesario tomar las precauciones necesarios al hacer los calculos.

In [ ]:
# Define diagnosticos para los problemas de salud de interes
FIBROSIS = ["E840", "E841", "E849", "E848"]

HIPERTENSION_PULMONAR = ["I270", "I272"]

FIBROSIS_PULMONAR = ["J841"]

ESCLERODERMIA = ["M348"]

MARFAN = ["Q874"]

EXCAVATUM = ["Q676"]

CARINATUM = ["Q677"]

SILICOSIS = ["J678", "J679"]

CARDIOPATIA_CONGENITA_ADULTO = []

TRASPLANTE_DE_PULMON = []

ECMO = []

CANCER_DE_PULMON = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D381",
]

CIRUGIA_CANCER_DE_PULMON = []

TRASPLNATE_DE_CORAZON = []

ENFERMEDAD_CORONARIA = []

ANEURISMA_DE_LA_AORTA = ["I712"]

DISECCION_DE_LA_AORTA = ["I710"]

MARCAPASOS = ["Z450"]

INSUFICIENCIA_MITRAL = ["I340"]

INSUFICIENCIA_AORTICA = ["I351"]

COVID = ["U071"]

APNEA_DEL_SUENO = ["G473"]

In [14]:
# Obtiene el resumen de pacientes y egresos por anio para el INT
cantidad_pacientes_por_diag = df_torax.group_by(pl.col(["ANO_EGRESO", "DIAG1"])).agg(
    n_pacientes=pl.col("ID_PACIENTE").n_unique(), n_egresos=pl.col("DIAG1").count()
)

In [16]:
cantidad_pacientes_por_diag.filter(pl.col("DIAG1").is_in()).pivot(values=pl.col("n_egresos"), index=pl.col(""))

ANO_EGRESO,DIAG1,n_pacientes,n_egresos
i64,str,u32,u32
2018,"""E849""",2,3
2016,"""E848""",13,23
2016,"""E840""",6,8
2015,"""E848""",10,19
2015,"""E840""",8,12
2017,"""E849""",5,5
2019,"""E848""",24,41
2019,"""E840""",10,12
2017,"""E848""",20,26
